# USDE College Scorecard: Data Science Project

## Prepare Data

### Packages

In [ ]:
# Basic imports for Data Science
import os
import numpy as np
import pandas as pd

# Visualization packages
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

# Geospatial analysis packages
import geopandas as gpd
import folium as fl
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


print("Setup complete!")

### Read Geographical Data

In [ ]:
# Read shapefile for all USA states into usa
usa = gpd.read_file("/kaggle/input/us-states-shapefile-from-census-gov-2024/tl_2024_us_state.shp")

# Filter USA to only include the Contiguous 48; Hawaii and Alaska filtered "manually" by "drop" method
con48 = usa[usa.REGION <= '4'].drop([31,40]).reset_index().copy()

# Set index as the name of the State & only keep the geometry column afterwards
con48map = con48[['STUSPS','geometry']].set_index('STUSPS').to_crs(epsg=4326)

### Read USDE Data

In [ ]:
# Read into a Pandas DataFrame a single 'csv' file from the USDE College Scorecard, e.g. most recent
# We save the 'usecols' parameter here in advance to allow us to use it again easily later on!
scoreCols = ['UNITID','OPEID','INSTNM','STABBR',
            'ZIP','LATITUDE','LONGITUDE','UGDS',
            'CURROPER','NPT4_PUB','NPT4_PRIV',
            'TUITIONFEE_IN','TUITIONFEE_OUT']
scorecard = pd.read_csv("/kaggle/input/usde-college-scoreboard-1996-2023/MERGED2022_23_PP.csv",
                       usecols=scoreCols, index_col='UNITID')

### Read US News TOP Universities

In [ ]:
top_univ = pd.read_csv("/kaggle/input/university-and-college-rankings-us-news/US-News-Rankings-Universities-Through-2023.csv",
                      usecols=['IPEDS ID','2023'], index_col='IPEDS ID')

# Remove rows without ranking, i.e. not TOP 150
top_univ.dropna(inplace=True)

# We don't really need the information of 2023 anymore, let's drop it too
top_univ.drop(['2023'], axis=1, inplace=True)

## Filter Top US Universities

The index of the scorecard dataset, which was set to `UNITID` when the file was read, is the same as `IPEDS ID` in the US News rankings file we imported. We will merge `scorecard` with `top_univ` to only filter the top ranked universities from our dataset.

In [ ]:
# Pandas `join` instead of `merge` chooses indeces by default
top_scorecard = scorecard.join(top_univ,how='inner').copy()
top_scorecard.info()

How does the distribution of tutitions looks like in public and private institutions?

In [ ]:
scorecard_public = top_scorecard[~top_scorecard.NPT4_PUB.isna()].copy()
scorecard_private = top_scorecard[~top_scorecard.NPT4_PRIV.isna()].copy()

In [ ]:
sns.boxplot(scorecard_public[['TUITIONFEE_IN','TUITIONFEE_OUT']])

In [ ]:
sns.boxplot(scorecard_private[['TUITIONFEE_IN']])

In [ ]:
sns.histplot(scorecard_public['TUITIONFEE_OUT']-scorecard_public['TUITIONFEE_IN'])

In [ ]:
sns.histplot(scorecard_public['TUITIONFEE_OUT']/scorecard_public['TUITIONFEE_IN'])

A few insights about the top universities we filtered:
1. Statistically, top public universities have the lowest tution rates for in-state applicants.
2. There is a wide distribution of out-of-state tuitions, from ~15K to ~55K USD.
3. The difference histogram shows most of these universities charge ~15-25K USD more for out-of-state students.
4. Looking at the ratio, we can see the majority double or triple (with a few charging quadruple more) for out-of-state students.
5. In many cases, the out-of-state tutition of the top public universities is not far from most private instutitions on this list.
6. With some exceptions/outliers, private institutions charge 50K-60K USD tutition.

The above data visualtizations show three outliers between the private institutions on that list, let's explore them. Easiest way--filter the DataFrame by the tuition variable, where it was smaller than 35K.

In [ ]:
scorecard_private[scorecard_private['TUITIONFEE_IN']<35000]

## Geographic Visualization

### Where are the TOP 150 universities in the US?
Next, we will visualize where are the TOP 150 we filtered before. We will use GeoPandas to convert the data frame & plot it on the map of the 48 states.

In [ ]:
# Use Folium (fl) to create an interactive map, centered on the 48 contiguous US states
m_1 = fl.Map(location=[39.8283,-98.5795],tiles='cartodbpositron', zoom_start=4)

# Create a Folium MarkerCluster object
mc = MarkerCluster()

# Create a marker for each institute in our list & add a popup with the institute name (INSTNM)
for idx, row in top_scorecard.iterrows():
    mc.add_child(Marker([row['LATITUDE'],
                         row['LONGITUDE']],
                        popup=fl.Popup(row['INSTNM'], max_width=100)))
m_1.add_child(mc)

# Display the map
m_1

#### Plot TOP 150 with a 2-colored legend, public and private
This visualization will help us understand which states provide the quality & affordable post-K12 education to their constituents.

In [ ]:
# Use Folium (fl) to create an interactive map, centered on the 48 contiguous US states
m_2 = fl.Map(location=[39.8283,-98.5795],tiles='cartodbpositron', zoom_start=4)

# If NPT4_PUB is >0, then it must be NaN; saving importing of math; val > 0 --> public institution
def color_producer(val):
    if val > 0:
        return 'blue'
    else:
        return 'darkred'

# Add a bubble map to the base map
for idx, row in top_scorecard.iterrows():
    Circle(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=500,
        color=color_producer(row['NPT4_PUB']),
        popup=row['INSTNM']).add_to(m_2)

# Display the map
m_2

#### Color states in two different maps, to indicate the cost considering only in-state or only out-state tuitions.
1. Looking at in-state can be useful for someone to decide where to live in the future, if he wishes to save a lot on tuition costs
2. But looking at out-state and private provides a better look, for example, for students arriving to the US from abroad, who seek to decide which state to move to for undergraduate studies.

Obviously, the latter case is incomplete. Many other factors going into such a decision. The easiest example is for a student who plans to work in a part-time job during his studies. Where would they earn more? That would depend on taxes, minimum wage, rent, etc. Purchasing power and cost of living differs not only between states, but also between cities (e.g., Buffalo vs. NYC are very different cases). But let's assume, for this case, that this tool could be useful not for a final decision making, but to provide some information on expected tutition costs.

However, we can also consider these a bit better if we visualize with the **cost of attendance** instead of **tuition** variable. We will visualize both below and see how it affects our point of view and perspective on the matter.

In [ ]:
# Create a public in-state tuition state-by-state summary
scorecard_instate = scorecard_public[['STABBR','TUITIONFEE_IN']].groupby(by="STABBR").mean()
scorecard_instate.shape

We grouped by the states and calculated the average tuition of the TOP 150 for each state. As expected, there are only 34 states in our new DataFrame, as not all 48 states have a top 150 public instutition in the TOP 150.

In [ ]:
# Create a base map
m_3 = fl.Map(location=[39.8283,-98.5795], tiles='cartodbpositron', zoom_start=4)

# Add a choropleth map to the base map
Choropleth(geo_data=con48map.__geo_interface__, 
           data=scorecard_instate['TUITIONFEE_IN'], 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Average In-State Tuition of Public Universities in US News Top 150 (2023)'
          ).add_to(m_3)

# Display the map
m_3

In [ ]:
# Create a state-by-state summary of all institutions, out-of-state tutition fee only
scorecard_outstate = top_scorecard[['STABBR','TUITIONFEE_OUT']].groupby(by="STABBR").mean()
scorecard_outstate.shape

In [ ]:
# Create a base map
m_4 = fl.Map(location=[39.8283,-98.5795], tiles='cartodbpositron', zoom_start=4)

# Add a choropleth map to the base map
Choropleth(geo_data=con48map.__geo_interface__, 
           data=scorecard_outstate['TUITIONFEE_OUT'], 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Average Out-of-State Tuition of US News Top 150 (2023)'
          ).add_to(m_4)

# Display the map
m_4